In [ ]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re


In [ ]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains

# Collecting statistics on forum communications in each product page